In [1]:
import torch
import numpy as np
from utils.dataset import SimpleIterDataset
import os

In [2]:
data_config_name = 'data/AK4/ak4_points_pf_sv_CHS_eta4p7.yaml'
import networks.particle_net_ak4_pf_sv as network_module
# export_onnx = "../sonic-models/models/particlenet_AK4/1/model.onnx"
export_onnx = "new_model.onnx"

In [3]:
model_state_dict = data_config_name.replace('.yaml','.pt')
jit_model_save = data_config_name.replace('.yaml','_ragged_gpu_jit.pt')
onnx_model = data_config_name.replace('.yaml','.onnx')

In [4]:
data_config = SimpleIterDataset([], data_config_name, for_training=False).config

K =  pf_points
K =  pf_features
K =  pf_mask
K =  sv_points
K =  sv_features
K =  sv_mask


In [5]:
data_config

In [6]:
model, model_info = network_module.get_model(data_config, for_inference=True)

In [7]:
model = torch.jit.script(model)

In [8]:
model_info

{'input_names': ['pf_points',
  'pf_features',
  'pf_mask',
  'sv_points',
  'sv_features',
  'sv_mask',
  'batch_count_pf_points',
  'batch_count_pf_features',
  'batch_count_pf_mask',
  'batch_count_sv_points',
  'batch_count_sv_features',
  'batch_count_sv_mask'],
 'input_shapes': {'pf_points': (100,),
  'pf_features': (1000,),
  'pf_mask': (50,),
  'sv_points': (10,),
  'sv_features': (55,),
  'sv_mask': (5,),
  'batch_count_pf_points': (1,),
  'batch_count_pf_features': (1,),
  'batch_count_pf_mask': (1,),
  'batch_count_sv_points': (1,),
  'batch_count_sv_features': (1,),
  'batch_count_sv_mask': (1,)},
 'output_names': ['softmax'],
 'dynamic_axes': {'pf_points': {0: 'n_pf_points'},
  'pf_features': {0: 'n_pf_features'},
  'pf_mask': {0: 'n_pf_mask'},
  'sv_points': {0: 'n_sv_points'},
  'sv_features': {0: 'n_sv_features'},
  'sv_mask': {0: 'n_sv_mask'},
  'batch_count_pf_points': {0: 'N'},
  'batch_count_pf_features': {0: 'N'},
  'batch_count_pf_mask': {0: 'N'},
  'batch_count_s

In [9]:
model.load_state_dict(torch.load(model_state_dict, map_location=torch.device('cpu')))

<All keys matched successfully>

In [10]:
model.eval()

RecursiveScriptModule(
  original_name=ParticleNetTagger
  (pf_conv): RecursiveScriptModule(
    original_name=FeatureConv
    (conv): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=BatchNorm1d)
      (1): RecursiveScriptModule(original_name=Conv1d)
      (2): RecursiveScriptModule(original_name=BatchNorm1d)
      (3): RecursiveScriptModule(original_name=ReLU)
    )
  )
  (sv_conv): RecursiveScriptModule(
    original_name=FeatureConv
    (conv): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=BatchNorm1d)
      (1): RecursiveScriptModule(original_name=Conv1d)
      (2): RecursiveScriptModule(original_name=BatchNorm1d)
      (3): RecursiveScriptModule(original_name=ReLU)
    )
  )
  (pn): RecursiveScriptModule(
    original_name=ParticleNet
    (edge_convs): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=EdgeConvBlock


In [11]:
device = 'cpu'
pf_points = torch.ones((8,), dtype = torch.float).to(device)
pf_features = torch.ones((80,), dtype = torch.float).to(device)
pf_mask = torch.ones((4,), dtype = torch.float).to(device)
sv_points = torch.ones((8,), dtype = torch.float).to(device)
sv_features = torch.ones((44,), dtype = torch.float).to(device)
sv_mask = torch.ones((4,), dtype = torch.float).to(device)

batch_pf_points = torch.tensor([2, 4, 6, 8]).to(device)
batch_pf_features = torch.tensor([20, 40, 60, 80]).to(device)
batch_pf_mask = torch.tensor([1, 2, 3, 4]).to(device)
batch_sv_points = torch.tensor([2, 4, 6, 8]).to(device)
batch_sv_features = torch.tensor([11, 22, 33, 44]).to(device)
batch_sv_mask = torch.tensor([1, 2, 3, 4]).to(device)

# batch_pf_points = torch.tensor([2, 6, 14]).to(device)
# batch_pf_features = torch.tensor([20, 60, 140]).to(device)
# batch_pf_mask = torch.tensor([1, 3, 7]).to(device)
# batch_sv_points = torch.tensor([2, 4, 6]).to(device)
# batch_sv_features = torch.tensor([11, 22, 33]).to(device)
# batch_sv_mask = torch.tensor([1, 2, 3]).to(device)

In [12]:
temp_inputs = (pf_points, pf_features, pf_mask, sv_points, sv_features, sv_mask, batch_pf_points, batch_pf_features, batch_pf_mask, batch_sv_points, batch_sv_features, batch_sv_mask)

In [13]:
print (model_info['input_names'])

['pf_points', 'pf_features', 'pf_mask', 'sv_points', 'sv_features', 'sv_mask', 'batch_count_pf_points', 'batch_count_pf_features', 'batch_count_pf_mask', 'batch_count_sv_points', 'batch_count_sv_features', 'batch_count_sv_mask']


In [14]:
model(*temp_inputs)

tensor([[7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02]], grad_fn=<SoftmaxBackward0>)

In [15]:
print(type(model))

<class 'torch.jit._script.RecursiveScriptModule'>


In [16]:
torch.onnx.export(model, temp_inputs, export_onnx,
                  input_names=model_info['input_names'],
                  output_names=model_info['output_names'],
                  dynamic_axes=model_info.get('dynamic_axes', None),
                  opset_version=15)

/Users/nirmalthomas/Library/Python/3.8/lib/python/site-packages/torch/onnx/symbolic_helper.py:719: UserWarning: allowzero=0 by default. In order to honor zero value in shape use allowzero=1
  warnings.warn("allowzero=0 by default. In order to honor zero value in shape use allowzero=1")


In [17]:
import onnxruntime as ort
# temp_path = "/Users/nirmalthomas/Desktop/CMS/model.onnx"
temp_path = export_onnx
ort_session = ort.InferenceSession(temp_path)

2022-05-27 00:28:55.513883 [W:onnxruntime:, constant_folding.cc:202 ApplyImpl] Unsupported output type of N11onnxruntime22SequenceTensorTypeBaseE. Can't constant fold SequenceEmpty node 'SequenceEmpty_384'
2022-05-27 00:28:55.528683 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '158'. It is not used by any node and should be removed from the model.
2022-05-27 00:28:55.528699 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '147'. It is not used by any node and should be removed from the model.
2022-05-27 00:28:55.528706 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '159'. It is not used by any node and should be removed from the model.
2022-05-27 00:28:55.528713 [W:onnxruntime:, graph.cc:3559 CleanUnusedInitializersAndNodeArgs] Removing initializer '160'. It is not used by any node and should be removed from the model.
2022-05-27 00:28:55.528719 [W:onnxruntime:, gr

In [18]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {
              'pf_points': to_numpy(pf_points),
              'pf_features': to_numpy(pf_features),
              'pf_mask': to_numpy(pf_mask),
              'sv_points': to_numpy(sv_points),
              'sv_features': to_numpy(sv_features),
              'sv_mask': to_numpy(sv_mask),
              'batch_count_pf_points': to_numpy(batch_pf_points),
              'batch_count_pf_features': to_numpy(batch_pf_features),
              'batch_count_pf_mask': to_numpy(batch_pf_mask),
              'batch_count_sv_points': to_numpy(batch_sv_points),
              'batch_count_sv_features': to_numpy(batch_sv_features),
              'batch_count_sv_mask': to_numpy(batch_sv_mask) 
             }

In [19]:
print ("ORT inputs - ", ort_inputs)

ORT inputs -  {'pf_points': array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), 'pf_features': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), 'pf_mask': array([1., 1., 1., 1.], dtype=float32), 'sv_points': array([1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), 'sv_features': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), 'sv_mask': array([1., 1., 1., 1.], dtype=float32), 'batch_count_pf_points': array([2, 4, 6, 8]), 'batch_count_pf_features': array([20, 40, 60, 80]), 'batch_count_pf_mask': array([1

In [20]:
runSessionParams = ort.RunOptions()
runSessionParams.log_severity_level = 0

In [21]:
ort_out = ort_session.run(None, ort_inputs, run_options = runSessionParams)

2022-05-27 00:28:55.652872 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.652943 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653051 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653118 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653149 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653178 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653234 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653273 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653305 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653332 [I:onnxruntime:, sequential_executor.cc:176 Execute] Begin execution
2022-05-27 00:28:55.653360 [I:onnxruntim

In [22]:
ort_out

[array([[7.1094264e-03, 1.4922640e-05, 4.3217994e-02, 1.1982591e-03,
         3.9007923e-01, 4.1971451e-01, 4.6418600e-02, 9.2247054e-02],
        [7.1094264e-03, 1.4922640e-05, 4.3217994e-02, 1.1982591e-03,
         3.9007923e-01, 4.1971451e-01, 4.6418600e-02, 9.2247054e-02],
        [7.1094264e-03, 1.4922640e-05, 4.3217994e-02, 1.1982591e-03,
         3.9007923e-01, 4.1971451e-01, 4.6418600e-02, 9.2247054e-02],
        [7.1094264e-03, 1.4922640e-05, 4.3217994e-02, 1.1982591e-03,
         3.9007923e-01, 4.1971451e-01, 4.6418600e-02, 9.2247054e-02]],
       dtype=float32)]

In [23]:
model(*temp_inputs)

tensor([[7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02],
        [7.1094e-03, 1.4923e-05, 4.3218e-02, 1.1983e-03, 3.9008e-01, 4.1971e-01,
         4.6419e-02, 9.2248e-02]], grad_fn=<SoftmaxBackward0>)

In [24]:
for index, item in enumerate(ort_session.get_inputs()):
    print ("Node # - {}".format(index))
    print ("Name - {}".format(item.name))
    print ("Shape - {}".format(item.shape))
    print ("Type - {}\n".format(item.type))

Node # - 0
Name - pf_points
Shape - ['n_pf_points']
Type - tensor(float)

Node # - 1
Name - pf_features
Shape - ['n_pf_features']
Type - tensor(float)

Node # - 2
Name - pf_mask
Shape - ['n_pf_mask']
Type - tensor(float)

Node # - 3
Name - sv_points
Shape - ['n_sv_points']
Type - tensor(float)

Node # - 4
Name - sv_features
Shape - ['n_sv_features']
Type - tensor(float)

Node # - 5
Name - sv_mask
Shape - ['n_sv_mask']
Type - tensor(float)

Node # - 6
Name - batch_count_pf_points
Shape - ['N']
Type - tensor(int64)

Node # - 7
Name - batch_count_pf_features
Shape - ['N']
Type - tensor(int64)

Node # - 8
Name - batch_count_pf_mask
Shape - ['N']
Type - tensor(int64)

Node # - 9
Name - batch_count_sv_points
Shape - ['N']
Type - tensor(int64)

Node # - 10
Name - batch_count_sv_features
Shape - ['N']
Type - tensor(int64)

Node # - 11
Name - batch_count_sv_mask
Shape - ['N']
Type - tensor(int64)



In [25]:
def create_ort_inputs(inputs, input_names):
    return {name: to_numpy(inputs[index]) for (index, name) in enumerate(input_names)}

In [26]:
def run_ort_inference(onnx_path, inputs, input_names):
    ort_session = ort.InferenceSession(onnx_path)
    runSessionParams = ort.RunOptions()
    runSessionParams.log_severity_level = 0
    ort_out = ort_session.run(None, create_ort_inputs(inputs, input_names), run_options = runSessionParams)
    return ort_out

In [27]:
def run_pt_inference(model, inputs):
    return model(*inputs)

In [28]:
def export_onnx(model, inputs, dynamic_axes, input_names, output_names, onnx_path, opset_version = 15):
    torch.onnx.export(model, inputs, onnx_path, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes, opset_version=opset_version)
    model_output = run_pt_inference(model, inputs)
    ort_output = run_ort_inference(onnx_path, inputs, input_names)
    print ("Model output - \n{}".format(model_output))
    print ("ORT output - \n{}".format(ort_output))

In [ ]:
def get_model(model_type):
    data_config_name = 'data/AK4/ak4_points_pf_sv_CHS_eta4p7.yaml'
    import networks.particle_net_ak4_pf_sv as network_module
    # export_onnx = "../sonic-models/models/particlenet_AK4/1/model.onnx"
    export_onnx = "new_model.onnx"